In [1]:
import pandas as pd
import neattext.functions as nfx
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel

In [2]:
df = pd.read_csv(r"C:\Users\HP\Desktop\my-mantine-app\Udemy_courses.csv")

In [3]:
df['clean_course_title'] = df['course_title'].apply(nfx.remove_stopwords)
df['clean_course_title'] = df['clean_course_title'].apply(nfx.remove_special_characters)

In [4]:
count_vect = CountVectorizer()

In [5]:
cv_mat = count_vect.fit_transform(df['clean_course_title'])

In [6]:
def recommend_course(df, course_descriptions, title, num_of_rec):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(course_descriptions)

    # Convert title and DataFrame course titles to lowercase for case-insensitive comparison
    title_lower = title.lower()
    df['course_title_lower'] = df['course_title'].str.lower()

    try:
        # Find the index of the course title (case-insensitive)
        course_index = df[df['course_title_lower'] == title_lower].index[0]
    except IndexError:
        print(f"Course '{title}' not found in the dataset.")
        return pd.DataFrame(columns=['course_title', 'similarity_score'])  # Return empty DataFrame
    finally:
        # Remove the temporary lowercase column
        df.drop(columns=['course_title_lower'], inplace=True)

    similarity_scores = cosine_similarity(tfidf_matrix, tfidf_matrix[course_index])
    
    sorted_indices = similarity_scores.argsort(axis=0)[::-1].flatten()[1:num_of_rec + 1]
    recommendations = [(df.iloc[i]['course_title'], similarity_scores[i][0]) for i in sorted_indices]
    
    recommended_courses = pd.DataFrame(recommendations, columns=['course_title', 'similarity_score'])
    return recommended_courses


In [9]:
course_descriptions = df['clean_course_title'].tolist()  # Assuming 'clean_course_title' column exists
title = "Autocad"  # Title of the course you want recommendations for (case-insensitive)
num_of_rec = 10

In [10]:
recommended_courses = recommend_course(df, course_descriptions, title, num_of_rec)
print(recommended_courses)

Course 'Autocad' not found in the dataset.
Empty DataFrame
Columns: [course_title, similarity_score]
Index: []
